In [4]:
import IEA_model
import numpy as np
import torch
import pandas as pd
from torch.utils.data import DataLoader
import utility

from importlib import reload 
reload(utility)
reload(IEA_model)

from sklearn.linear_model import LinearRegression

In [5]:
thold = 1e-3

df = pd.read_csv("gene_list_PCA/THOLD_{}.txt".format(thold))
idx_gene = df["index"].to_numpy()

data_train = utility.image_gene_dataset(train = True, cv = None)
data_test = utility.image_gene_dataset(train = False, cv = None)

train_loader = DataLoader(data_train)
test_loader = DataLoader(data_test)


sid_train, features_train, genes_train = data_train[:]
sid_test, features_test, genes_test = data_test[:]

sid_P1, features_P1 = utility.load_SSL_features_P1()
features_P1 = torch.tensor(features_P1)

/ocean/projects/asc170022p/juc91/anaconda/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/ocean/projects/asc170022p/juc91/anaconda/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


(9179, 581, 128)

In [7]:
M.forward?

In [6]:
beta = .1
gamma = 1.0
L = 2

Z_train_cv = []
Z_test_cv = []
Z_P1_cv = []


M = IEA_model.IEA( L = L,  idx_gene = idx_gene ).cuda()

for cv in range(5):
    filename = "models/model_L{}_cv{}_beta{}_gamma{}_genes{}".format(L, cv, beta, gamma, len(idx_gene))

    M.load_state_dict( torch.load(filename) )
    
    with torch.no_grad():
        _, _, _, Z_train = M.forward( features_train.cuda(), non_prb = True )
        Z_train_cv.append( Z_train.cpu().data.numpy() )
        
        _, _, _, Z_test = M.forward( features_test.cuda(), non_prb = True  )
        Z_test_cv.append( Z_test.cpu().data.numpy() )

        _, _, _, Z_P1 = M.forward( features_P1.cuda(), non_prb = True  )
        Z_P1_cv.append( Z_P1.cpu().data.numpy() )
        
Z_train_cv = np.array( Z_train_cv )
Z_test_cv = np.array( Z_test_cv )
Z_P1_cv = np.array( Z_P1_cv )
    

In [70]:
Z_P1.shape

torch.Size([9179, 2])

In [7]:
Z_train_aligned = []
Z_test_aligned = []
Z_P1_aligned = []

for cv in range(0,5):
    Z_cat = np.concatenate( [ Z_train_cv[ 0, :, : ], Z_train_cv[ cv, :, : ] ], axis = 1 )
    corrcoef = np.corrcoef(Z_cat.T)
    
    print(corrcoef)
    print()
    
    Z_train = []
    Z_test = []
    Z_P1 = []
    for lll in range(L):
        
        
        corr_lll = corrcoef[lll, L:]
        idx_lll = np.argmax( np.abs(corr_lll) )
        
        
        mean = Z_train_cv[cv, :, idx_lll].mean()
        std = Z_train_cv[cv, :, idx_lll].std()
        
        if corr_lll[idx_lll]>0:
            Z_train.append( ( Z_train_cv[cv, :, idx_lll] - mean ) / std )
            Z_test.append( ( Z_test_cv[cv, :, idx_lll] - mean ) / std  )
            Z_P1.append(( Z_P1_cv[cv, :, idx_lll] - mean ) / std  )
        else:
            Z_train.append( - ( Z_train_cv[cv, :, idx_lll] - mean) / std )
            Z_test.append( - ( Z_test_cv[cv, :, idx_lll] - mean) / std )
            Z_P1.append( - ( Z_P1_cv[cv, :, idx_lll] - mean ) / std  )
        
    
    Z_train_aligned.append( np.array(Z_train).T )
    Z_test_aligned.append( np.array(Z_test).T )
    Z_P1_aligned.append( np.array(Z_P1).T )
    
Z_train_aligned = np.array(Z_train_aligned)
Z_test_aligned = np.array(Z_test_aligned)
Z_P1_aligned = np.array(Z_P1_aligned)


[[ 1.         -0.03789817  1.         -0.03789817]
 [-0.03789817  1.         -0.03789817  1.        ]
 [ 1.         -0.03789817  1.         -0.03789817]
 [-0.03789817  1.         -0.03789817  1.        ]]

[[ 1.         -0.03789817  0.97762419  0.0302235 ]
 [-0.03789817  1.         -0.13281486  0.96053367]
 [ 0.97762419 -0.13281486  1.         -0.02186219]
 [ 0.0302235   0.96053367 -0.02186219  1.        ]]

[[ 1.         -0.03789817  0.05340738  0.98937643]
 [-0.03789817  1.          0.98183229 -0.02769713]
 [ 0.05340738  0.98183229  1.          0.07147242]
 [ 0.98937643 -0.02769713  0.07147242  1.        ]]

[[ 1.         -0.03789817  0.98900336 -0.06416191]
 [-0.03789817  1.         -0.02435581  0.96476894]
 [ 0.98900336 -0.02435581  1.         -0.03764199]
 [-0.06416191  0.96476894 -0.03764199  1.        ]]

[[ 1.         -0.03789817  0.13037045 -0.97666967]
 [-0.03789817  1.         -0.91316037  0.06960488]
 [ 0.13037045 -0.91316037  1.         -0.10591901]
 [-0.97666967  0.069604

In [8]:
Z_train_final = Z_train_aligned.mean(0)
Z_test_final = Z_test_aligned.mean(0)
Z_P1_final = Z_P1_aligned.mean(0)

In [9]:
df_train = pd.DataFrame(data = Z_train_final, columns = ["IEA{}".format(iii) for iii in range(L) ], index = sid_train)
df_test = pd.DataFrame(data = Z_test_final, columns = ["IEA{}".format(iii) for iii in range(L) ], index = sid_test)
df_P1 = pd.DataFrame(data = Z_P1_final, columns = ["IEA{}".format(iii) for iii in range(L) ], index = sid_P1)



In [10]:
df_train.to_csv("output/IEA_train.csv", index_label = "sid")
df_test.to_csv("output/IEA_test.csv", index_label = "sid")
df_P1.to_csv("output/IEA_P1.csv", index_label = "sid")